In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-02-25 23:24:11--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-02-25 23:24:12 (6.06 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("GT-FinalProject").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# import and clean data frame
from pyspark import SparkFiles
url="https://aws-gt-finalproject001.s3.amazonaws.com/full_esrb_rating_data.csv"
spark.sparkContext.addFile(url)
rating_data_df = spark.read.option('header', 'true').csv(SparkFiles.get("full_esrb_rating_data.csv"), inferSchema=True, sep=',')
rating_data_df = rating_data_df.drop('_c0')
rating_data_df = rating_data_df.dropDuplicates(["title"])
rating_data_df.show(20)

+--------------------+-------+-----------------+--------------+-----+--------------+----------------+-----------+--------------+----------------+----------------+--------+------+------------+----------+---------------------+---------------------+-------------+-----------+----------------------+-------------+--------------+------+--------------+--------------+-------------+------------------+---------------+---------------------+-----------------+--------------+------------------------+--------+-----------+
|               title|console|alcohol_reference|animated_blood|blood|blood_and_gore|cartoon_violence|crude_humor|drug_reference|fantasy_violence|intense_violence|language|lyrics|mature_humor|mild_blood|mild_cartoon_violence|mild_fantasy_violence|mild_language|mild_lyrics|mild_suggestive_themes|mild_violence|no_descriptors|nudity|partial_nudity|sexual_content|sexual_themes|simulated_gambling|strong_language|strong_sexual_content|suggestive_themes|use_of_alcohol|use_of_drugs_and_alcoho

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://aws-gt-finalproject.c9qveamcfmcm.us-east-2.rds.amazonaws.com:5432/esrb_rating_data"
config = {"user":"postgres",
          "password": "awsdbpassword001",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

rating_data_df.write.jdbc(url=jdbc_url, table='esrb_rating', mode=mode, properties=config)